In [12]:
import json
import os
import pandas as pd
import numpy as np
import scipy.stats
from scipy import stats, ndimage
from scipy.signal import butter, lfilter, medfilt

#Change these for the data

trialLength = 3000
baselineLength = 1000

In [13]:
def get_series_from_control(control_data, device_id, check_field, substr, retrieve_field):
    return list(map(lambda x: x[retrieve_field],
                    list(filter(lambda x: substr in x[check_field] if check_field in x else False, control_data[device_id]))
                   ))

def get_continuous_frames_block_with_timestamps(unity_data, start_ts, end_ts):
    frames = unity_data
    ret = []
    
    for frame in frames:
        if frame['timestamp'] > end_ts:
            return ret
        elif frame['timestamp'] >= start_ts:
            ret.append(frame)
    return ret
    

In [14]:
def process_participant(participantID):

    path = "/Users/dhewitt/OneDrive - Nexus365/Data/TTC/"
    
    CONTROL_FILE_NAME = os.path.join(path + participantID + "/" + participantID + "__Control_Data__.json")
    UNITY_FILE_NAME = os.path.join(path + participantID + "/" + participantID + "__Device_Data_Unity_C_Sharp_Client_VR__.json")
    PKL_FILE_NAME = os.path.join(path + participantID + "/" + participantID + "_dataframelist.pkl")

    control_data = None
    with open(CONTROL_FILE_NAME) as f:
        control_data = json.load(f)

    #Getting the timestamps for each trial

    UNITY_DEVICE_ID = "o8Y6VNWF7orzDfPGCrJh"

    start_timestamps = get_series_from_control(control_data, UNITY_DEVICE_ID, 'start_new_trial_ttc', '', 'timestamp')

    start_timestamps = [x - baselineLength for x in start_timestamps]

    unity_data = None
    with open(UNITY_FILE_NAME) as f:
        unity_data = json.load(f)

    frame_chunks_by_start_timestamps = [get_continuous_frames_block_with_timestamps(unity_data, timestamp, timestamp + baselineLength + trialLength) 
                                        for timestamp in start_timestamps]

    df = pd.DataFrame(frame_chunks_by_start_timestamps) #the data is a list: trials x data within each trial. 

    #Finding the part of the data we want from the list, and making a df from that
    dataframe_list = list(map(lambda x: pd.DataFrame(x), frame_chunks_by_start_timestamps))
    
    pd.to_pickle(dataframe_list, PKL_FILE_NAME)
    
 

In [15]:
# List of participant IDs
participant_ids = ["P01"]  # Add more participant IDs as needed

# Loop over each participant
for participantID in participant_ids:
    # ... (The existing code for processing a single participant will go here)
    process_participant(participantID)